In [ ]:
import keras.backend as K
import numpy as np
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.layers import Input
from matplotlib.pyplot import imshow, show, imsave
from time import time

In [ ]:
style_image = "style/image/path"
content_image = "content/image/path"

In [ ]:
def preprocess(image_path):
    img = image.load_img(image_path, target_size=(600,800))
    img = image.img_to_array(img)
    img = preprocess_input(img)
    img = np.expand_dims(img, axis=0)
    return img

In [ ]:
def deprocess_image(x):
    x = x[0]
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [ ]:
style_image=preprocess(style_image)
content_image = preprocess(content_image)

In [ ]:
gen_image = K.variable(np.random.randn(1,600,800,3))

In [ ]:
print("style_image ",style_image.shape)
print("content_image ",content_image.shape)
print("gen_image ",gen_image.shape)

In [ ]:
style_layers = [1,4,7,11]
content_layers = [12]

In [ ]:
model = VGG16(include_top=False, weights='imagenet', input_tensor=Input(tensor=gen_image))
model.summary()

In [ ]:
style_features = []
for layer in style_layers:
    fn = K.function([model.layers[0].input], [model.layers[layer].output])
    style_features.append(fn([style_image])[0])

In [ ]:
content_features = []
for layer in content_layers:
    fn = K.function([model.layers[0].input], [model.layers[layer].output])
    content_features.append(fn([content_image])[0])    

In [ ]:
for _ in style_features:
    print(_.shape)

In [ ]:
content_features[0].shape

In [ ]:
m, w, h, c = a_C.shape()
a_C_unrolled = K.reshape(content_features[0], (-1,w*h,c))
del content_features
print("a_C_unrolled ", a_C_unrolled.shape)

In [ ]:
def gram_matrix(A):
    return K.dot(A, K.transpose(A))

In [ ]:
def content_cost():
    J_content = K.variable(0.)
    
    a_G = model.layers[12].output
    m, n_H, n_W, n_C = a_G.get_shape().as_list()
    a_G_unrolled = K.reshape(a_G,(-1, n_H*n_W, n_C))
    
    J_content = (1/(4*n_H*n_W*n_C))*K.sum(K.square(a_C_unrolled-a_G_unrolled))
    return J_content

In [ ]:
def compute_layer_style_cost(a_S, a_G):
    
    J_style_layer = K.variable(0.)
    m, n_H, n_W, n_C = a_G.get_shape().as_list()
   
    a_S = K.transpose(K.reshape(a_S, (n_H*n_W,n_C)))
    a_G = K.transpose(K.reshape(a_G, (n_H*n_W,n_C)))

    GS = gram_matrix(a_S)
    GG = gram_matrix(a_G)


    J_style_layer = K.square(1/(2*n_C*n_H*n_W))*K.sum(K.sum(K.square(GS-GG)))
    
    return J_style_layer

In [ ]:
def compute_style_cost(model=model, STYLE_LAYERS=style_layers):
    
    J_style = K.variable(0.)
    i=0

    for layer in STYLE_LAYERS:
        
        a_G = model.layers[layer].output

        J_style_layer = compute_layer_style_cost(style_features[i], a_G)

        J_style += 0.25 * J_style_layer
        i+=1

    return J_style

In [ ]:
def total_cost(alpha = 1000, beta = 4):

    J = K.variable(0.)
    J = (alpha*content_cost()+beta*compute_style_cost())
    
    return J

In [ ]:
total_loss = total_cost()

In [ ]:
total_loss

In [ ]:
gen_image

In [ ]:

opt = Adam(lr=2.5)
updates = opt.get_updates([gen_image],[],total_loss)
train = K.function([],[total_loss],updates)
print("Training Step defined.")
start = time()
for epoch in range(500):
    e_start = time()
    out = train([])
    e_end = time()
    print("Epoch: {}, Loss: {:.2e}, Style Loss: {:.2e}, Content Loss: {:.2e}, Time taken per Step: {:.2f}".format(epoch, out[0],out[1],out[2], e_end-e_start))
    if(epoch%20==0):
        imshow(deprocess_image(K.get_value(gen_image)))
        show()
        print("ETA: {}".format((e_end-start)*(500-epoch)/(epoch+1)))